# 6. Бинарные файлы: Pickle

## ЭТАП 1

Обучим модель линейной регрессии на встроенном датасете Diabetes dataset. Опустим кросс-валидацию и разделение выборки на обучение и тест, так как для наших целей это не важно.

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
regressor = LinearRegression()
regressor.fit(X,y)

LinearRegression()

Объект regressor теперь является обученной моделью. 

## ЭТАП 2

Импортируем модуль и воспользуемся методом dumps:

In [2]:
import pickle
model = pickle.dumps(regressor)
type(model), type(regressor)

(bytes, sklearn.linear_model._base.LinearRegression)

Как видим, мы создали объект model типа bytes. 

## ЭТАП 3

Восстановим объект Python:

In [3]:
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes

LinearRegression()

## ЭТАП 4
Сохраним объект в файл

In [4]:
with open('myfile.pkl', 'wb') as output:
       pickle.dump(regressor, output)

Теперь у нас есть бинарный файл с готовой моделью.

## ЭТАП 5

Файл myfile.pkl так же легко десериализовать:

In [5]:
with open('myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file

LinearRegression()

## ЭТАП 6
Убедимся, что методы и результаты предсказаний обученной модели и загруженной из файла совпадают:

In [7]:
all(regressor.predict(X) == regressor_from_bytes.predict(X))

True

In [6]:
all(regressor.predict(X) == regressor_from_file.predict(X))

True

## ОГРАНИЧЕНИЯ

Как мы упоминали, у Pickle есть ограничения. Например, мы не сможем сериализовать лямбда-функции. Давайте посмотрим, что нам вернёт следующий код:

In [8]:
my_lambda = lambda x: x*2
with open('my_lambda.pkl', 'wb') as output:
    pickle.dump(my_lambda, output)

PicklingError: Can't pickle <function <lambda> at 0x0000024A1348A4C8>: attribute lookup <lambda> on __main__ failed

### Совет. В таких случаях лучше пользоваться пакетом [dill](https://github.com/uqfoundation/dill).

## Задание 6.1

Что делает метод dumps из библиотеки Pickle?

- позволяет сохранить сериализованный объект в файл
- возвращает сериализованный объект верно
- загружает объект из файла и десериализует его
- загружает объект из потока байт

## Задание 6.2
Что делает метод load из библиотеки Pickle?

- позволяет сохранить сериализованный объект в файл
- возвращает сериализованный объект
- загружает объект из файла и десериализует его верно
- загружает объект из потока байт

# 7. Сохранение пайплайна

Мы уже упоминали, что Pickle работает с любыми объектами Python. Поэтому для сохранения может быть доступна не просто обученная модель, но и целый пайплайн, включающий предобработку данных. 

In [10]:
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from joblib import dump, load


X, y = load_diabetes(return_X_y=True)

pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])


pipe.fit(X, y)
pipe.predict(X[1:2])
s1 = pickle.dumps(pipe)
pipe_from_bytes = pickle.loads(s1)

print(all(pipe.predict(X) == pipe_from_bytes.predict(X)))

True


Задание 7.1
Что выдаст этот код?

True

Обратите внимание, что для работы с файлами используются методы pickle.dump и pickle.load, а для работы с сериализованными объектами pickle.dumps и pickle.loads.

Как мы видим, Pickle прекрасно справляется со своей задачей, однако иногда массивы данных бывают настолько большими, что после загрузки из Pickle можно не восстановить объект полностью. 

В таких случаях лучше использовать Joblib вместо Pickle. Этот модуль более эффективен для объектов, которые содержат большие массивы данных. Он оптимизирован для быстрой и надежной работы с данными большого объема. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой.

Для иллюстрации работы сохраним обученную модель:

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
regressor = LinearRegression()
regressor.fit(X, y)

LinearRegression()

In [13]:
from joblib import dump, load
dump(regressor, 'regr.joblib')

['regr.joblib']

In [14]:
#…и загрузим его заново:
clf_from_jobliv = load('regr.joblib') 
all(regressor.predict(X) == clf_from_jobliv.predict(X))

True

# 8. Практика: Pickle

## ЗАДАНИЕ 1
Коллега обучил модель и теперь просит вас проверить её на ваших данных. Он присылает вам Pickle-файл. Загрузите модель, используя модуль Pickle. (hw1.pkl)

## Вопрос 8.1
При загрузке вывелся секретный код. Введите его в поле ниже.

In [19]:
with open('hw1.pkl', 'rb') as pkl_file:
    secret_model = pickle.load(pkl_file)
secret_model

secret word: skillfactory
how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg


LinearRegression()

## Вопрос 8.2

Проверьте, какого типа объект получился. Введите название библиотеки, которую использовал ваш коллега.

In [20]:
type(secret_model)

sklearn.linear_model._base.LinearRegression

Ответ: sklearn

Вопрос 8.3

Теперь вам нужно применить модель. Сделайте предсказание для этого набора фичей: 
\[1, 1, 1, 0.661212487096872]'. Введите результат (округлите до тысячных, например 0.853).

In [26]:
X = [[1, 1, 1, 0.661212487096872]]

In [27]:
secret_model.predict(X)

array([0.666])

## ЗАДАНИЕ 2

У модели есть два поля с именами a и b. Создайте из них словарь с такими же именами ключей и значениями, а затем сохраните в файл с помощью модуля Pickle. 

Проверить, что вы всё сделали правильно, можно с помощью скрипта.

In [33]:
#!python hw1_check_ol.py mydict.pkl

In [36]:
my_dict = {'a':secret_model.a, 'b':secret_model.b}

In [37]:
with open('my_dict.pkl', 'wb') as output:
       pickle.dump(my_dict, output)

In [38]:
!python hw1_check_ol.py my_dict.pkl

('secret code 2:', '3c508')


# 9. Форматы PMML и ONNX-ML

In [39]:
!pip install nyoka==4.2.1

In [40]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Тренировка пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipe.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipe.pmml")

## *Задание для продвинутых (необязательно, но рекомендуется)

Для выполнения этого задания вам понадобится ознакомиться с документацией.

В задаче ниже мы обучаем модель sklearn, конвертируем ее в ONNX и делаем инференс через ONNX-runtime.

Дополните код ниже недостающими элементами:

In [45]:
import onnxruntime as rt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#from skl2onnx import ___1____

#from skl2onnx.common.data_types import ___2___
from skl2onnx.common.data_types import FloatTensorType

# загружаем данные
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
#model.fit(___3___, y_train)
model.fit(X_train, y_train)

# делаем инференс моделью на тесте
#test_pred = model.predict(___4___)
test_pred = model.predict(X_test)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в onnx формат
# initial_type = [('float_input', ___5___([None, ___6___]))]
initial_type = [('float_input', FloatTensorType([None, 10]))]

#model_onnx = ___7___(model, initial_types=initial_type)
model_onnx = convert_sklearn(model, initial_types=initial_type)

# сохраняем модель в файл
with open("model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через onnxruntime
#sess = rt.___8___("model.onnx")
sess = rt.InferenceSession("model.onnx")

input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx) 

ImportError: cannot import name 'get_all_providers' from 'onnxruntime.capi._pybind_state' (D:\Anaconda3\lib\site-packages\onnxruntime\capi\_pybind_state.py)